In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00


In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install soyspacing -q

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: casperjy0000. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(project='project-test', reinit=True)

wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_054924-45j4y25s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run absurd-snowflake-3
wandb: ⭐️ View project at https://wandb.ai/casperjy0000/project-test
wandb: 🚀 View run at https://wandb.ai/casperjy0000/project-test/runs/45j4y25s


In [ ]:
!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

fatal: destination path 'korean-malicious-comments-dataset' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from transformers.integrations import WandbCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device =torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
print("device", device)

device cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/corrected_daily_vanishing_2_with_dailyBank_1.csv', encoding='utf-8') # 대략 일상데이터 9000, 보이스피싱 1400
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aug_data.csv', encoding='utf-8') # 대략 일상데이터 9000, 보이스피싱 2400
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/finished_data.csv', encoding='utf-8') # aug_data + 일상 금융 대화 14000건

df.head()

,transcript,label
0,안녕하세요~^^ 20대 여성입니다!! 안녕하세요~ 저도 20대 여성입니다~ 반가워요...,0
1,앱에서 송금했던 계좌번호 조회가 되나요. 네. 고객님. 조회 가능합니다. 어디에서 ...,0
2,나는 인제 선물에 관해서 꼭 잘 챙기는 법인데 뭐 누구 생일이라든가 뭐 무슨 데이라...,0
3,혹시 가족 관계가 어떻게 되세요? 네. 저는 마흔일곱 남편이라고 하지만 가장 큰 아...,0
4,이번에 기사를 하나 봤는데 여. 폐지 줍는 노인 절반 분들이 월에 10만 원도 못 ...,0


In [ ]:
df['label'].value_counts()

0    9191
1    2639
Name: label, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11830 entries, 0 to 11829
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   transcript  11830 non-null  object
 1   label       11830 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 185.0+ KB


# 학습 데이터 나누기

In [ ]:
# train_data = df.sample(frac=0.8, random_state=42)
# vaild_test_data = df.drop(train_data.index)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['transcript'], df['label'], test_size=0.2, shuffle=True, random_state=777, stratify=df['label'])

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle=True, random_state=777, stratify=y_train)

In [ ]:
print(len(x_train))
print(len(x_test))
print(len(x_val))

7098
2366
2366


In [ ]:
train_data = pd.concat([x_train, y_train], axis=1)
test_data = pd.concat([x_test, y_test], axis=1)
valid_data = pd.concat([x_val, y_val], axis=1)


In [ ]:
# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터 : {}'.format(len(test_data)))
print('중복 제거 전 검증 데이터 : {}'.format(len(valid_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["transcript"], inplace=True)
test_data.drop_duplicates(subset=["transcript"], inplace=True)
valid_data.drop_duplicates(subset=["transcript"], inplace=True)

# 데이터 갯수 확인
print('중복 제거 후 학습 데이터 :  {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터 : {}'.format(len(test_data)))
print('중복 제거 후 검증 데이터 : {}'.format(len(valid_data)))

중복 제거 전 학습 데이터 : 7098
중복 제거 전 테스트 데이터 : 2366
중복 제거 전 검증 데이터 : 2366
중복 제거 후 학습 데이터 :  7098
중복 제거 후 테스트 데이터 : 2366
중복 제거 후 검증 데이터 : 2366


#토크나이징

In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenizer.save_pretrained('tokenizer_KcELECTRA')

('tokenizer_KcELECTRA/tokenizer_config.json',
 'tokenizer_KcELECTRA/special_tokens_map.json',
 'tokenizer_KcELECTRA/vocab.txt',
 'tokenizer_KcELECTRA/added_tokens.json',
 'tokenizer_KcELECTRA/tokenizer.json')

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["transcript"]),
    return_tensors = "pt",
    max_length = 256,
    padding = True,
    truncation = True,
    add_special_tokens=True,
)

In [ ]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '안녕', '##하세요', '~', '저는', '60세', '여자', '##입니다', '.', '반가', '##워요', '.', '안녕', '##하세요', '?', '저는', '40대', '남자', '##이고', '강원도', '##에', '살아요', '.', '요즘', '날씨', '##가', '부쩍', '추워', '##졌', '##던데', '가을', '##이', '이제', '오', '##려나', '##봐요', '.', '그러네', '##요', '.', '제가', '사는', '강원도', '##도', '정말', '추워', '##졌어', '##요', '.', '강원도', '##하면', '설악산', '##이', '유명', '##하잖아요', '.', '설악산', '##에', '단풍', '##도', '곧', '많이', '들', '##겠어요', '.', '맞아요', '.', '가을', '##이', '##되면', '단풍', '##관광', '##철이', '##라', '사람이', '많아요', '.', '그렇', '##겠어요', '.', '단풍', '구경', '##하러', '오는', '관광객', '##들이', '북적', '##북', '##적', '하겠네', '##요', '.', '관광객', '##이', '많아서', '복잡', '##한데', '저는', '서늘', '##한', '날씨', '##를', '싫어', '##해요', '.', '그럼', '가을', ',', '겨울', '##이', '싫', '##으시', '##겠네요', '.', '저는', '가을', ',', '겨울', '##을', '제일', '좋아하', '##거든요', '.', '그러', '##시', '##군요', '.', '저는', '가을', '겨울', '##이라는', '계절', '##이', '가끔

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data["transcript"]),
    return_tensors = "pt",
    max_length = 256,
    padding = True,
    truncation = True,
    add_special_tokens=True,
)

In [ ]:
print(tokenized_test_sentences[0])
print(tokenized_test_sentences[0].tokens)
print(tokenized_test_sentences[0].ids)
print(tokenized_test_sentences[0].attention_mask)

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '울산', '##페이', '결제', '##가', '잘', '되', '##다가', '갑자기', '왜', '안될까요', '?', '어떤', '결제', '시', '그러', '##셨나요', '?', 'Q', '##R', '##을', '했', '##거든요', '.', '어떤', '매장', '##을', '이용', '##하셨', '##죠', '?', 'OO', '##마트', '##에서', '##요', '.', 'OO', '##마트', '##가', '울산', '##가', '##맹', '##점이', '아닐', '수', '있습니다', '.', '그럼', '결제', '##가', '아예', '안되요', '?', '울산', '##페이', '체크카드', '##를', '신청', '##하시면', '이용', '##하실', '수', '있습니다', '.', '가맹점', '##이', '아니더라도', '그게', '있으면', '되는', '거예요', '?', '네', '.', '고객', '##님', '.', 'Q', '##R', '##결', '##제', '##만', '원하는', '곳도', '있던데', ',', '뭐가', '다르', '##나요', '?', '네', ',', '다릅니다', '.', '어떤게', '##요', '?', '체크카드', '##는', '연결', '##된', '카드', '##사에', '수수료', '##를', '지급', '##하지만', 'Q', '##R', '##은', '수수료', '##를', '내지', '않습니다', '.', '와', ',', '그런', '차이가', '있었', '##군요', '.', '울산', '##페이', '결제', '##하면', '년', '##말', '##정', '##산', '##할', '때', '소득', '##공', '##제', '되

In [ ]:
tokenized_valid_sentences = tokenizer(
    list(valid_data["transcript"]),
    return_tensors = "pt",
    max_length = 256,
    padding = True,
    truncation = True,
    add_special_tokens=True,
)

In [ ]:
print(tokenized_valid_sentences[0])
print(tokenized_valid_sentences[0].tokens)
print(tokenized_valid_sentences[0].ids)
print(tokenized_valid_sentences[0].attention_mask)

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '안녕', '##하세요', '!', '21', '##시간', '##만이', '##네요', '어', '##젠', '뭐', '##하셨어요', 'ㅎㅎ', '?', '저는', '어제', '근처에', '베이커리', '##에서', '빵', '##을', '좀', '사', '##왔', '##어요', '.', '뭐하고', '지내', '##셨어요', '?', '전', '유행', '##이', '좀', '지나', '##긴', '했지만', '집에서', '달고', '##나', '##를', '만들어', '##봤어', '##요', '음', '.', '.', '재밌', '##기만', '하지', '맛', '##은', '잘', '모르겠', '##어요', '아', '달고', '##나', '##를', '만드', '##셨', '##구나', '얼마전에', '오징어게임', '##때문에', '달고', '##나가', '다시', '각광', '##받는', '것', '같', '##더라구요', '.', '단', '##걸', '원래', '안좋아', '##하시나요', '?', '맞아요', '초콜릿', '##은', '좋아하는', '##데', '그냥', '단', '##걸', '안좋아', '##하는', '것', '같아요', '전주', '##에는', '디저트', '##로', '고구마', ',', '밤', '이런', '구', '##황', '##작물', '디저트', '##가', '유행', '##인가봐', '##요', '근데', '전', '구', '##황', '##작물', '##도', '싫어', '##해요', '아', '정말', '##요', '?', '왜', '싫어하', '##세요', '?', '?', '저는', '왠지', '그', '.', '.', '뭔가', '짜고', '매운', '이런', '맛',

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


In [ ]:
train_label = train_data["label"].values
test_label = test_data["label"].values
valid_label = valid_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)
valid_dataset = CurseDataset(tokenized_valid_sentences, valid_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/Colab Notebooks/',
    num_train_epochs= 3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    save_strategy='steps',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    report_to="wandb",
    load_best_model_at_end=True
)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score(labels, preds)
  wandb.log({"accuracy": acc})
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
  }

In [ ]:
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1), WandbCallback()],
)

You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback
EarlyStoppingCallback


In [ ]:
trainer.train()

<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.091000,0.024158,0.994083,0.986564,1.000000,0.973485
200,0.018600,0.001053,0.999577,0.999052,1.000000,0.998106
300,0.013900,0.003439,0.999577,0.999052,1.000000,0.998106
400,0.000600,0.003413,0.999577,0.999052,1.000000,0.998106
500,0.006500,0.003091,0.999577,0.999052,1.000000,0.998106
600,0.000300,0.003139,0.999577,0.999052,1.000000,0.998106


<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=600, training_loss=0.02182663546875119, metrics={'train_runtime': 4608.0757, 'train_samples_per_second': 4.621, 'train_steps_per_second': 0.145, 'total_flos': 2524287465123840.0, 'train_loss': 0.02182663546875119, 'epoch': 2.7})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/model_log/test_model_directory1")

In [ ]:
results = trainer.evaluate(eval_dataset=test_dataset)
wandb.log({"test_loss": results["eval_loss"], "test_accuracy": results["eval_accuracy"]})

<ipython-input-26-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# from soyspacing.countbase import CountSpace
# soy_model = CountSpace()

# corpus_fname = '/content/drive/MyDrive/Colab Notebooks/demo_model_134963_norm.txt'
# soy_model.train(corpus_fname)

all tags length = 694236 --> 57795, (num_doc = 15602)

In [ ]:
from soyspacing.countbase import CountSpace
model_fname = '/content/drive/MyDrive/Colab Notebooks/space-correct_wiki.model'
soy_model = CountSpace()
soy_model.load_model(model_fname, json_format=False)

In [ ]:
# sentence = input('')
# corrected_sentence = soy_model.correct(sentence)
# print()
# print('before: ',sentence)
# print("after: ", corrected_sentence[0])

In [ ]:
import torch.nn.functional as F

def sentence_predict(sent):
    model.eval()

    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=256,
    )

    tokenized_sent = tokenized_sent.to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"],
        )

    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)

    return probabilities

while True:
    sentence = input("문장을 입력해주세요 ('0'을 입력하면 종료): ")
    if sentence == "0":
        break
    corrected_sentence = soy_model.correct(sentence)
    probabilities = sentence_predict(corrected_sentence[0])

    # 클래스 레이블 대신 확률 값을 출력
    labels = ["일반인", "피싱범"]
    for label, prob in zip(labels, probabilities[0]):
        print(f"{label}: {prob:.4f}")

    # 가장 높은 확률을 가진 클래스를 예측 결과로 출력
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    prediction = labels[predicted_class]
    print(f"예측 결과: {prediction}\n")

문장을 입력해주세요 ('0'을 입력하면 종료): 안녕하세요, XX은행 고객지원센터입니다. 현재 모바일뱅킹 앱에 보안 업데이트가 필요하다고 나오는데 확인해보셨나요? 아니요, 무슨 문제가 있나요? 최근에 보안 이슈가 발생하여 업데이트가 필요합니다. 귀하의 정보를 보호하기 위해 본인 확인이 필요해요. 계좌번호와 생년월일을 알려주실 수 있을까요?"
일반인: 0.9989
피싱범: 0.0011
예측 결과: 일반인



KeyboardInterrupt: ignored